### Title:
__Exact simulation of the 3/2 model__

### Authors:
* Jan Baldeaux

### Abstract:
This paper discusses the exact simulation of the stock price process underlying the 3/2 model. Using a result derived by Craddock and Lennox using Lie Symmetry Analysis, we adapt the Broadie-Kaya algorithm for the simulation of affine processes to the 3/2 model. We also discuss variance reduction techniques and find that conditional Monte Carlo techniques combined with quasi-Monte Carlo point sets result in significant variance reductions.

### Link:
* [Journal](https://www.worldscientific.com/doi/abs/10.1142/S021902491250032X)
* [arXiv](http://arxiv.org/abs/1105.3297)

### References
* Baldeaux J (2012) Exact simulation of the 3/2 model. Int J Theor Appl Finan 15:1250032. https://doi.org/10.1142/S021902491250032X
* Kouarfate IR, Kouritzin MA, MacKay A (2021) Explicit Solution Simulation Method for the 3/2 Model. In: Hernández‐Hernández D, Leonardi F, Mena RH, Pardo Millán JC (eds) Advances in Probability and Mathematical Statistics. Springer International Publishing, Cham, pp 123–145

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import time
import scipy.stats as spst
import scipy.special as spsp
import copy 
#import mp math as mp

import sys
sys.path.insert(sys.path.index('')+1, 'D:/Github/pyfeng')
import pyfeng as pf
import pyfeng.ex as pfex
np.set_printoptions(precision=4)

Run only one below

## Case I: Eq. (4.2) in Baldeaux (2012)

In [3]:
mr, theta, vov, rho, sigma, intr = 2.0, 1.5, 0.2, -0.5, 1.0, 0.05
strike, spot, texp = 1, 1, 1
p_exact = 0.443059

## Set 2 in Kouarfate et al. (2021)

In [4]:
sigma, mr, theta, vov, rho = 0.06, 22.84, 0.218, 8.56, -0.99
intr = 0

strike, spot, texp = np.array([95, 100, 105]), 100, 0.2
p_exact = np.array([10.364, 7.386, 4.938])
iv_exact = pf.Bsm(1).impvol(p_exact, strike, spot, texp)
iv_exact * 100

array([43.7918, 41.4578, 39.2219])

## Set 5 in Kouarfate et al. (2021)

In [5]:
sigma, mr, theta, vov, rho = 0.06, 20.48, 0.218, 3.20, -0.99
intr = 0

strike, spot, texp = np.array([95, 100, 105]), 100, 0.5
p_exact = np.array([11.724, 8.999, 6.710])
iv_exact = pf.Bsm(1).impvol(p_exact, strike, spot, texp)
iv_exact * 100

array([32.8172, 31.9686, 31.1591])

## Pricing with Time Discreteization 

In [6]:
m = pfex.Sv32McTimeStep(sigma, vov, rho, mr, theta, intr)
m.set_num_params(n_path=1.6e5, dt=1/50, rn_seed=123456)
m.scheme = 2  # Exact Stepping with 1 / NCX2
m.correct_fwd = False

In [7]:
%%time
bias = m.price(strike, spot, texp) - p_exact
print(bias)

[-0.0202 -0.0126 -0.0054]
CPU times: total: 234 ms
Wall time: 239 ms


## Pricing with Exact Simulation

In [8]:
m1 = pfex.Sv32McBaldeaux2012Exact(sigma, vov, rho, mr, theta, intr)
m1.set_num_params(n_path=10000, rn_seed=123456, dt=None)
m1.correct_fwd = False

In [9]:
%%time
bias = m.price(strike, spot, texp) - p_exact
print(bias)  #-0.051878003020602914  -0.051878003020602914

[-0.0227 -0.0145 -0.0086]
CPU times: total: 219 ms
Wall time: 233 ms


## Pricing with IG approximation (Almost Exact Simulation)

In [12]:
m = pfex.Sv32McChoiKwok2023Ig(sigma, vov, rho, mr, theta, intr)
m.set_num_params(n_path=10000, rn_seed=123456, dt=1/10)
m.correct_fwd = False

In [13]:
%%time
bias = m.price(strike, spot, texp) - p_exact
print(bias)

4.999999999999999 [54.8507 58.2898 76.6823 ... 64.8276 64.5719 64.184 ]


AttributeError: module 'numpy' has no attribute 'float'.
`np.float` was a deprecated alias for the builtin `float`. To avoid this error in existing code, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations